In [58]:

#imports --
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import os.path
import json
import seaborn as sns
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.rdFingerprintGenerator import GetRDKitFPGenerator
import ot
import csv
from pickle import dump
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import pairwise_distances
from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid
from sklearn.datasets import make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, RationalQuadratic

In [34]:
#import EMD calculation from Jiale's project
from Polymer_Similarity import EMD_Calculation_pot as EMD
D_test = EMD(
            query_smiles_list=['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO'],
            query_smiles_weight_list=[1,1],
            target_smiles_list=['C(=O)CC(=O)', 'Oc1ccc(cc1)C(C)(C)c2ccc(cc2)O'],
            target_smiles_weight_list=[1,1],
            embedding_function="RDKFingerprint",
            similarity_score_function="Tanimoto",
            radius = 5,
            num_bits = 512,
            )

print(D_test)

def str_to_list(string):
    counter=string.count(',')
    #print(counter)
    list=[]
    for i in range(counter):
        if counter ==1:
            string=string[1:len(string)-1]
        if i==0 and counter>1:
            j=string.index(',')
            list+=[string[2:j-1]]
            string=string[j+1:len(string)-1]
            if string[0]==' ':
                string=string[1:]
        elif i==counter-1:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[j+1]==' ':
                list+=[string[j+3:len(string)-1]]
            else:
                list+=[string[j+2:len(string)-1]]
        else:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[0]==' ':
                string=string[j+2:]
            else:
                string=string[j+1:]
        #print(string)

    return list

print(str_to_list("['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']"))


0.8087301587301587
['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']


In [3]:
#write summary document for original library data
rates=pd.read_csv('./all_rates.csv')
#print(rates)
ogpolym=pd.read_csv('./all_reactions_monomers.csv',header=0)
monomers=pd.read_csv('Monomer Info_DMTDMI.csv',header=0)
reducedmon=monomers['SMILES_Monomers'].tolist()
with open('full_original_library.csv', 'w', newline='') as file:
    file_writer=csv.writer(file, delimiter=',')
    file_writer.writerow(['Name','SMILES_List','Ratio_List','Biodegradability'])
with open('reduced_original_library.csv', 'w', newline='') as file:
    file_writer=csv.writer(file, delimiter=',')
    file_writer.writerow(['Name','SMILES_List','Ratio_List','Biodegradability'])
for i in range(len(ogpolym.index)):
    name=ogpolym['reaction'][i]
    SMILES_list=[ogpolym['monomer1'][i],ogpolym['monomer2'][i]]
    Ratios=[ogpolym['ratio1'][i],ogpolym['ratio2'][i]]
    try:
        index=rates['reaction'].tolist().index(name)
    except:
        continue
    if rates['rate'][index] ==0:
        Bio=0
    else:
        Bio=1
    if ogpolym['monomer1'][i] in reducedmon or ogpolym['monomer1flip'][i] in reducedmon:
        if ogpolym['monomer2'][i] in reducedmon or ogpolym['monomer2flip'][i] in reducedmon:
            with open('reduced_original_library.csv', 'a', newline='') as file:
                file_writer=csv.writer(file, delimiter=',')
                file_writer.writerow([name,SMILES_list,Ratios,Bio])
    with open('full_original_library.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([name,SMILES_list,Ratios,Bio])


In [3]:
#write mL split files with necessary information
ogsplit0=pd.read_csv('./ML_splits/datasplit_all_0.csv',header=0)
ogsplit1=pd.read_csv('./ML_splits/datasplit_all_1.csv',header=0)
ogsplit2=pd.read_csv('./ML_splits/datasplit_all_2.csv',header=0)
ogsplit3=pd.read_csv('./ML_splits/datasplit_all_3.csv',header=0)
ogsplit4=pd.read_csv('./ML_splits/datasplit_all_4.csv',header=0)
ogsplit5=pd.read_csv('./ML_splits/datasplit_all_5.csv',header=0)
alldata=pd.read_csv('./full_original_library.csv',header=0)
set=[ogsplit0,ogsplit1,ogsplit2,ogsplit3,ogsplit4,ogsplit5]



for i in range(len(set)):
    list=set[i]['reaction'].tolist()
    mask=alldata['Name'].isin(list)
    data=alldata[mask]
    #save to a file
    data.to_csv(path_or_buf='./ML_splits/original_split'+str(i)+'.csv')


In [6]:
#generate EMD representations for the datasplit sets against the entire library
#use RDKfingerprint, radius 5, numbits=512, similarity tanimoto
os0=pd.read_csv('./ML_splits/original_split0.csv',header=0)
os1=pd.read_csv('./ML_splits/original_split1.csv',header=0)
os2=pd.read_csv('./ML_splits/original_split2.csv',header=0)
os3=pd.read_csv('./ML_splits/original_split3.csv',header=0)
os4=pd.read_csv('./ML_splits/original_split4.csv',header=0)
os5=pd.read_csv('./ML_splits/original_split5.csv',header=0)
alldata=pd.read_csv('./full_original_library.csv',header=0)
set=[os0,os1,os2,os3,os4,os5]
for k in range(len(set)):
    col=[]
    for i in range(len(set[k].index)):
        row=[]
        for j in range(len(alldata.index)):
            X=set[k]['SMILES_List'][i]
            print(X)
            X=str_to_list(X)
            Y=set[k]['Ratio_List'][i]
            Z=alldata['SMILES_List'][j]
            Z=str_to_list(Z)
            W=alldata['Ratio_List'][j]
            #W=str_to_list(W)
            print(X, Y, Z ,W)
            score=EMD(
                query_smiles_list=X,
                query_smiles_weight_list=Y,
                target_smiles_list=Z,
                target_smiles_weight_list=W,
                embedding_function="RDKFingerprint",
                similarity_score_function="Tanimoto",
                radius = 5,
                num_bits = 512,
                )
            row+=[score]
        col+=[row]
    #save the list of lists as a npy array
    formed=np.array(col)
    np.save('./ML_splits/EMD_fororiginal_split'+str(k)+'_vsfull_library.npy',formed,allow_pickle=False)


In [33]:
#load all the numpy arrays with the representations and the decisions
set0=np.load('./ML_splits/EMD_fororiginal_split0_vsfull_library.npy')
set1=np.load('./ML_splits/EMD_fororiginal_split1_vsfull_library.npy')
set2=np.load('./ML_splits/EMD_fororiginal_split2_vsfull_library.npy')
set3=np.load('./ML_splits/EMD_fororiginal_split3_vsfull_library.npy')
set4=np.load('./ML_splits/EMD_fororiginal_split4_vsfull_library.npy')
set5=np.load('./ML_splits/EMD_fororiginal_split5_vsfull_library.npy')
y_set0=np.load('./ML_splits/Biodeg_fororiginal_split0_vsfull_library.npy')
y_set1=np.load('./ML_splits/Biodeg_fororiginal_split1_vsfull_library.npy')
y_set2=np.load('./ML_splits/Biodeg_fororiginal_split2_vsfull_library.npy')
y_set3=np.load('./ML_splits/Biodeg_fororiginal_split3_vsfull_library.npy')
y_set4=np.load('./ML_splits/Biodeg_fororiginal_split4_vsfull_library.npy')
y_set5=np.load('./ML_splits/Biodeg_fororiginal_split5_vsfull_library.npy')

def concatenate_indices(indices,leave_out):
    #indices is a list of the split arrays, leave out is the index number of the test set
    newlist=[]
    test=None
    for i in range(len(indices)):
        if i==leave_out:
            test=indices[i]
        else:
            newlist+=[indices[i]]
    train=newlist[0]
    for i in range(1,len(newlist)):
        train=np.concatenate((train,newlist[i]),0)
    return [train,test]

X_list = [set0, set1, set2, set3, set4, set5]
Y_list = [y_set0, y_set1, y_set2, y_set3, y_set4, y_set5]

xs=concatenate_indices(X_list,None)
print(xs)


[array([[0.80873016, 0.25609756, 0.        , ..., 0.91583333, 0.90201613,
        0.90201613],
       [0.59795086, 0.46654275, 0.46992481, ..., 0.9125    , 0.86803746,
        0.86803746],
       [0.41671865, 0.8581309 , 0.85436017, ..., 0.90021008, 0.85939153,
        0.85939153],
       ...,
       [0.66319444, 0.66516854, 0.64016854, ..., 0.86261261, 0.83444444,
        0.83444444],
       [0.8559744 , 0.74149844, 0.73914294, ..., 0.870436  , 0.84363918,
        0.84363918],
       [0.79464286, 0.7625    , 0.7375    , ..., 0.41666667, 0.4125    ,
        0.4125    ]]), None]


In [27]:
####write homemade cross validation function for retraining the base model
X_list = [set0, set1, set2, set3, set4, set5]
Y_list = [y_set0, y_set1, y_set2, y_set3, y_set4, y_set5]


def rf_cross_val(C, xs, ys):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,max_leaf_nodes=C[2])
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)
    return [mean, std]


C = [2 ** 7, 4, 16]

print(rf_cross_val(C, X_list, Y_list))


[0.7256324715040311, 0.03240013808049135]


In [14]:
#optimize the original library across different parameters
number_estimators = [2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10]
max_depth = [1,2,4,8,16,32,64,128,256]
min_samples_split = [2]
min_samples_leaf = [1]
max_leaf_nodes = [2,4,8,16,32,64,128,256,512]
#make a list of all the combinations of parameters to run
combos=[]
for i in number_estimators:
    for j in max_depth:
        for k in max_leaf_nodes:
            combos+=[[i,j,k]]
#print(combos)

#get the trained values and write them into a file to save parameters, cross val score, and error
with open('original_library_training_optimization_rf.csv', 'w', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Number of Estimators','Max Depth','Min Samples Split','Min Samples Leaf','Max Leaf Nodes','Mean Cross Val Score','STD'])

for i in range(len(combos)):
    com=combos[i]
    output=rf_cross_val(com,X_list,Y_list)
    with open('original_library_training_optimization_rf.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([com[0],com[1],2,1,com[2],output[0],output[1]])
    print('done combo ',str(i))



done combo  0
done combo  1
done combo  2
done combo  3
done combo  4
done combo  5
done combo  6
done combo  7
done combo  8
done combo  9
done combo  10
done combo  11
done combo  12
done combo  13
done combo  14
done combo  15
done combo  16
done combo  17
done combo  18
done combo  19
done combo  20
done combo  21
done combo  22
done combo  23
done combo  24
done combo  25
done combo  26
done combo  27
done combo  28
done combo  29
done combo  30
done combo  31
done combo  32
done combo  33
done combo  34
done combo  35
done combo  36
done combo  37
done combo  38
done combo  39
done combo  40
done combo  41
done combo  42
done combo  43
done combo  44
done combo  45
done combo  46
done combo  47
done combo  48
done combo  49
done combo  50
done combo  51
done combo  52
done combo  53
done combo  54
done combo  55
done combo  56
done combo  57
done combo  58
done combo  59
done combo  60
done combo  61
done combo  62
done combo  63
done combo  64
done combo  65
done combo  66
done 

In [22]:
#train and make reduced library RF model

#first make and save random splits (groups of 6)
#get the degrading subset
all_reduced=pd.read_csv('./reduced_original_library.csv')
#we're separating out the degrading and non-degrading portions so that they are evenly distributed across all the splits (all splits have equal fractions degrading and non-degrading). Library was randomly generated so doing so for the chemical structures doesn't make as much sense
degrading_reduced=all_reduced.loc[all_reduced["Biodegradability"]==1]
nondegrading_reduced=all_reduced.loc[all_reduced["Biodegradability"]==0]
num=6
for i in range(num):
     fraction=1/(num-i)
     degrading=degrading_reduced.sample(frac= fraction)
     degrading_reduced=degrading_reduced.drop(degrading.index)
     nondegrading=nondegrading_reduced.sample(frac=fraction)
     nondegrading_reduced=nondegrading_reduced.drop(nondegrading.index)
     dataframe=pd.concat([degrading,nondegrading])
     #print(dataframe)
     dataframe.to_csv(path_or_buf='./Reduced_ML_Splits/reduced_split'+str(i)+'.csv')

         Name                                        SMILES_List Ratio_List  \
23    DW_10-1                          ['C(=O)COCC(=O)', 'OCCO']     [1, 1]   
104    DW_6-6                    ['C(=O)CCCC(=O)', 'OCCC(C)CCO']     [1, 1]   
136     R_116                ['C(=O)CCCC(=O)', 'OCC(CC1)CCC1CO']     [1, 1]   
195      R_26             ['C(=O)CCCCCCCCC(=O)', 'OC(CC1)CCC1O']     [1, 1]   
95     DW_5-7                     ['C(=O)CCC(=O)', 'OCCC(C)CCO']     [1, 1]   
66    DW_17-6                ['C(=O)COCC(=O)', 'OCC(CC1)CCC1CO']     [1, 1]   
58    DW_16-6                ['C(=O)CSCC(=O)', 'OCC(CC1)CCC1CO']     [1, 1]   
9       AC_16  ['C(=O)CCCCCCCCC(=O)', 'Oc1ccc(cc1)C(C)(C)c2cc...     [1, 1]   
155      R_15                      ['C(=O)CSCC(=O)', 'OCC(C)CO']     [1, 1]   
102    DW_6-4                       ['C(=O)CCCC(=O)', 'OCCCCCO']     [1, 1]   
232      R_91                ['C(=O)C(CC1)CCC1C(=O)', 'OCC(C)O']     [1, 1]   
18      AC_55  ['C(=O)CCCCCCCC(=O)', 'Oc1ccc(cc1)C(C

In [35]:
#get the emd scores for each of the subsets vs the entire library
rl0=pd.read_csv('./Reduced_ML_Splits/reduced_split0.csv',header=0)
rl1=pd.read_csv('./Reduced_ML_Splits/reduced_split1.csv',header=0)
rl2=pd.read_csv('./Reduced_ML_Splits/reduced_split2.csv',header=0)
rl3=pd.read_csv('./Reduced_ML_Splits/reduced_split3.csv',header=0)
rl4=pd.read_csv('./Reduced_ML_Splits/reduced_split4.csv',header=0)
rl5=pd.read_csv('./Reduced_ML_Splits/reduced_split5.csv',header=0)
rl_all=pd.concat([rl0,rl1,rl2,rl3,rl4]).reset_index()


set_rl=[rl0,rl1,rl2,rl3,rl4,rl5]
training=[]
for k in range(len(set_rl)):
    col=[]
    for i in range(len(set_rl[k].index)):
        row=[]
        for j in range(len(rl_all.index)):
            X=set_rl[k]['SMILES_List'][i]
            #print(X)
            X=str_to_list(X)
            Y=set_rl[k]['Ratio_List'][i]
            Z=rl_all['SMILES_List'][j]
            Z=str_to_list(Z)
            W=rl_all['Ratio_List'][j]
            #W=str_to_list(W)
            #print(X, Y, Z ,W)
            score=EMD(
                query_smiles_list=X,
                query_smiles_weight_list=Y,
                target_smiles_list=Z,
                target_smiles_weight_list=W,
                embedding_function="RDKFingerprint",
                similarity_score_function="Tanimoto",
                radius = 5,
                num_bits = 512,
                )
            row+=[score]
        col+=[row]
    #save the list of lists as a npy array
    formed=np.array(col)
    training+=[formed]
    np.save('./Reduced_ML_Splits/EMD_forreduced_split'+str(k)+'_vsfull_reduced.npy',formed,allow_pickle=False)
test=training[5]
training=training[0:5]



In [36]:
#extract the biodegradability column info for the reduced library and save as dataframe
trainingy=[]
for i in range(5):
    sample=set_rl[i]
    nparray=sample["Biodegradability"].to_numpy()
    np.save('./Reduced_ML_Splits/biodeg_forreduced_split'+str(i)+'_vsfull_reduced.npy',formed,allow_pickle=False)
    trainingy+=[nparray]
testy=set_rl[5]["Biodegradability"].to_numpy()


In [39]:
def rf_cross_val(C, xs, ys,testx,testy):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,max_leaf_nodes=C[2])
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)

    #now evaluate against unseen testing data
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,
                                   max_leaf_nodes=C[2])
    model.fit(X[0], Y[0])
    eval=model.score(testx,testy)
    with open("./240523_Optimization_Models/RF_"+str(testx)[0:2]+"_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std, eval]

print(rf_cross_val([4,4,4],training,trainingy,test,testy))

[0.7356410256410256, 0.0917733823865145, 0.9230769230769231]


In [50]:
def get_EMD_array(sets,basis,EMD_values=["RDKFingerprint","Tanimoto",5,512]):
    #sets = list of sets (pd dataframes) that need EMD calculations against the basis
    #if basis=None then the concatenation of the sets is used as basis. If not, give polymer basis
    #EMD_values gives [embedding_function, similarity_score_function, radius, num_bits]

    EMD_arrays=[]
    for k in range(len(sets)):
        col = []
        for i in range(len(sets[k].index)):
            row = []
            for j in range(len(basis.index)):
                X = sets[k]['SMILES_List'][i]
                #print(X)
                X = str_to_list(X)
                Y = sets[k]['Ratio_List'][i]
                #print(basis['SMILES_List'])
                #print("basis",basis['SMILES_List'][j])
                Z = basis['SMILES_List'][j]
                #print(Z)
                Z = str_to_list(Z)
                W = basis['Ratio_List'][j]
                # W=str_to_list(W)
                #print(X, Y, Z, W)
                score = EMD(
                    query_smiles_list=X,
                    query_smiles_weight_list=Y,
                    target_smiles_list=Z,
                    target_smiles_weight_list=W,
                    embedding_function=EMD_values[0],
                    similarity_score_function=EMD_values[1],
                    radius=EMD_values[2],
                    num_bits=EMD_values[3],
                )
                row += [score]
            col += [row]
        # save the list of lists as a npy array
        formed = np.array(col)
        EMD_arrays+=[formed]
    return EMD_arrays
 #first we need to get the optimization done for the full original library and reduced original library
 #load all the data for the original library
print("running the Original EMD calcs")
os0=pd.read_csv('./ML_splits/original_split0.csv',header=0)
os1=pd.read_csv('./ML_splits/original_split1.csv',header=0)
os2=pd.read_csv('./ML_splits/original_split2.csv',header=0)
os3=pd.read_csv('./ML_splits/original_split3.csv',header=0)
os4=pd.read_csv('./ML_splits/original_split4.csv',header=0)
os5=pd.read_csv('./ML_splits/original_split5.csv',header=0)
y_set0=np.load('./ML_splits/Biodeg_fororiginal_split0_vsfull_library.npy')
y_set1=np.load('./ML_splits/Biodeg_fororiginal_split1_vsfull_library.npy')
y_set2=np.load('./ML_splits/Biodeg_fororiginal_split2_vsfull_library.npy')
y_set3=np.load('./ML_splits/Biodeg_fororiginal_split3_vsfull_library.npy')
y_set4=np.load('./ML_splits/Biodeg_fororiginal_split4_vsfull_library.npy')
y_set5=np.load('./ML_splits/Biodeg_fororiginal_split5_vsfull_library.npy')
os_train=[os0,os1,os2,os3,os4] #the training data used for cross validation
#print("concatenated training")
# #print(pd.concat(os_train,axis=0).reset_index())
os_train_emd=get_EMD_array(os_train,pd.concat(os_train).reset_index())
os_train_ys=[y_set0,y_set1,y_set2,y_set3,y_set4]
os_test_emd=get_EMD_array([os5],pd.concat(os_train).reset_index())
os_test_ys=y_set5
 ###### now load and set up the EMD scoring for the reduced library

print("running the reduced EMD calcs")
rl0=pd.read_csv('./Reduced_ML_Splits/reduced_split0.csv',header=0)
rl1=pd.read_csv('./Reduced_ML_Splits/reduced_split1.csv',header=0)
rl2=pd.read_csv('./Reduced_ML_Splits/reduced_split2.csv',header=0)
rl3=pd.read_csv('./Reduced_ML_Splits/reduced_split3.csv',header=0)
rl4=pd.read_csv('./Reduced_ML_Splits/reduced_split4.csv',header=0)
rl5=pd.read_csv('./Reduced_ML_Splits/reduced_split5.csv',header=0)

rl_train=[rl0,rl1,rl2,rl3,rl4] #the training data used for cross validation
rl_train_emd=get_EMD_array(rl_train,pd.concat(rl_train).reset_index())
#load the y values
rl_train_ys=[]
for i in range(5):
    sample=rl_train[i]
    nparray=sample["Biodegradability"].to_numpy()
    rl_train_ys+=[nparray]
rl_test_emd=get_EMD_array([rl5],pd.concat(rl_train).reset_index())
rl_test_ys=rl5["Biodegradability"].to_numpy()


running the Original EMD calcs
running the reduced EMD calcs


In [51]:
losses=['hinge','log_loss','perceptron']
alphas=[0.0001,0.001,0.01,0.1,1,10,100]
import
combos_SDG=[]
for i in losses:
    for j in alphas:
        combos_SDG+=[[i,j]]

#get the trained values for full library and write them into a file to save parameters, cross val score, and error
with open('original_library_training_optimization_SDG.csv', 'w', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Loss Function','Alpha','Mean Cross Val Score','STD','Test Evaluation Score'])
#get the trained values and write them into a file to save parameters, cross val score, and error
with open('reduced_library_training_optimization_SDG.csv', 'w', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Loss Function','Alpha','Mean Cross Val Score','STD','Test Evaluation Score'])



In [59]:
def lin_cross_val(C, xs, ys,xtest,ytest,name):
    #C should be [loss, alpha]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model =SGDClassifier(loss=C[0], alpha=C[1],shuffle=True)
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True)
    model.fit(X[0],Y[0])
    eval=model.score(xtest,ytest)
    #save the model
    with open("./240523_Optimization_Models/SGD_"+str(name)+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std,eval]

def train_lin(C, xs,ys):
    X = concatenate_indices(xs, i)
    Y = concatenate_indices(ys, i)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True)
    model.fit(X[0], Y[0])
    return model

def update_lin(old_model, C, xs, ys):
    #xs are additional new data that needs to be fit
    #ys are labels corresponding to xs
    class_biodeg=[0,1] #pretty sure since we use 0 or 1 this is all we've got
    model=old_model.partial_fit(xs,ys,classes=class_biodeg)
    return model

def task2(arg):
    com=arg
    # execute across original library
    output = lin_cross_val(com, os_train_emd, os_train_ys, os_test_emd[0], os_test_ys,'os')
    with open('original_library_training_optimization_SDG.csv', 'a', newline='') as file:
        file_writer = csv.writer(file, delimiter=',')
        file_writer.writerow([com[0], com[1], output[0], output[1], output[2]])
    # execute across reduced library
    output = lin_cross_val(com, rl_train_emd, rl_train_ys, rl_test_emd[0], rl_test_ys,'rl')
    with open('reduced_library_training_optimization_SDG.csv', 'a', newline='') as file:
        file_writer = csv.writer(file, delimiter=',')
        file_writer.writerow([com[0], com[1], output[0], output[1], output[2]])
    print('done combo ', str(com), flush=True)



In [60]:
for i in combos_SDG:
    task2(i)

done combo  ['hinge', 0.0001]
done combo  ['hinge', 0.001]
done combo  ['hinge', 0.01]
done combo  ['hinge', 0.1]
done combo  ['hinge', 1]
done combo  ['hinge', 10]
done combo  ['hinge', 100]
done combo  ['log_loss', 0.0001]
done combo  ['log_loss', 0.001]
done combo  ['log_loss', 0.01]
done combo  ['log_loss', 0.1]
done combo  ['log_loss', 1]
done combo  ['log_loss', 10]
done combo  ['log_loss', 100]
done combo  ['perceptron', 0.0001]
done combo  ['perceptron', 0.001]
done combo  ['perceptron', 0.01]
done combo  ['perceptron', 0.1]
done combo  ['perceptron', 1]
done combo  ['perceptron', 10]
done combo  ['perceptron', 100]
